![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

<br>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CLASSIFICATION_PATIENT_COMPLAINT_AND_PATIENT_URGENCY.ipynb)

## **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.1.0
Spark NLP_JSL Version : 5.1.0


# 🔎 MODELS

<div align="center">

| **Index** | **Question Answer Models**        |
|---------------|----------------------|
| 1        | [few_shot_classifier_patient_complaint_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/09/04/few_shot_classifier_patient_complaint_sbiobert_cased_mli_en.html)     |
| 2          | [bert_sequence_classifier_patient_complaint](https://nlp.johnsnowlabs.com/2023/09/04/bert_sequence_classifier_patient_complaint_en.html)       |
| 3          | [genericclassifier_patient_complaint_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/09/04/genericclassifier_patient_complaint_sbiobert_cased_mli_en.html)       |



</div>

# few_shot_classifier_patient_complaint_sbiobert_cased_mli

In [ ]:
text_list = [
"""My experience in the emergency room was one of the worst of my life. After arriving with severe stomach pain, I was made to wait hours to see anyone despite unbearable cramping. The nurse who finally took me back was completely dismissive, barely examining me before declaring it gastroenteritis. But the pain worsened until I was shrieking in agony. Only then did a doctor properly evaluate me and determine my appendix had ruptured. Their delayed diagnosis resulted in a dangerous infection leaving me hospitalized for weeks and missing work. Neither the nurse nor doctor apologized or admitted their lack of thoroughness almost cost my life. To make matters worse, they discharged me with little instruction and no follow-up care. This kind of negligent, substandard ER practice is unacceptable and I plan to report it before someone dies next time""",
"""I cannot properly express how much our family appreciates everything the hospital ICU staff did for my father when COVID-19 nearly killed him. The compassionate nurses worked tirelessly to care for him as he struggled to breathe for weeks on the ventilator. Doctors provided transparent updates to us and adjusted his treatment until he finally turned the corner. Therapists later helped him slowly regain strength and mobility. After narrowly surviving COVID thanks to the amazing care here, my father is home with us again. We are overwhelmed with gratitude for the ICU team who never gave up on him even when the prognosis seemed dire. They are true heroes who we credit with saving his life. We will be forever grateful for their dedication, skill and humanity.""",
"""My appointment was not only a complete waste of my time, but the dermatologist's dismissive attitude caused me distress. I arrived on time for my biopsy results, but had to wait over 45 minutes past my slot because they overbooked patients. When the doctor finally saw me, he barely glanced at my chart before declaring the growth benign. He said nothing else and rushed out, leaving me bewildered. His brusque manner left me feeling like an inconvenience. Later I realized he had not even removed the growth discussed at my previous visit. His haste and lack of careful examination put my health at risk. I called to address these issues with the office manager but she was defensive and made no effort to rectify the situation. I will not be returning to a practice that cares so little about attentive patient care.""",
"""The nursing staff on the orthopedic ward were exceptional. They explained all my medications and answered questions thoroughly without making me feel rushed. When post-op pain flared, they responded quickly to make me more comfortable, even during busy shifts. The nurses helped my worried family feel welcome and let them stay close by as I recovered. Their compassionate bedside manner eased my anxiety. With their assistance, I was soon moving about safely. The nurses went the extra mile, bringing me extra pillows, distraction devices and magazines. Their skill and attentiveness helped me get back on my feet faster than expected. I cannot say enough positive things about my nursing care! Each team member provided kind, professional support throughout my hip replacement recovery. I wish I could name them all to recognize their stellar care.""",
"""I was shocked to receive a bill for over $5,000 after a routine doctor’s appointment because the office staff submitted the claim improperly. They insisted the expensive labs and scans were done despite my insistence nothing beyond a physical exam occurred. When I asked them to investigate further, they were rude and accusatory, claiming I must be lying about what procedures I had. Even after I provided date-stamped documentation proving the billing was wrong, they refused to correct the error unless I paid a portion of the amount upfront. It caused me untold stress fighting to get this fraudulent claim removed from my record. The office manager was entirely unhelpful when I raised concerns about their unethical billing practices. Patients should not have their financial security threatened by careless administrative mistakes covered up by staff. I will be reporting them to the authorities and insurance commission for their negligent business practices and harassment of clients like myself""",
]

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

few_shot_classifier = FewShotClassifierModel.pretrained("few_shot_classifier_patient_complaint_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

pipeline_model = clf_Pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = LightPipeline(pipeline_model)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
few_shot_classifier_patient_complaint_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'prediction')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+--------------+
|                                                                                                text|    prediction|
+----------------------------------------------------------------------------------------------------+--------------+
|[My experience in the emergency room was one of the worst of my life. After arriving with severe ...|   [Complaint]|
|[I cannot properly express how much our family appreciates everything the hospital ICU staff did ...|[No_Complaint]|
|[My appointment was not only a complete waste of my time, but the dermatologist's dismissive atti...|   [Complaint]|
|[The nursing staff on the orthopedic ward were exceptional. They explained all my medications and...|[No_Complaint]|
|[I was shocked to receive a bill for over $5,000 after a routine doctor’s appointment because the...|   [Complaint]|
+-------------------------------------------------------

# bert_sequence_classifier_patient_complaint

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_patient_complaint", "en", "clinical/models")\
    .setInputCols(["document",'token'])\
    .setOutputCol("prediction")

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier])

pipeline_model = clf_Pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
light_model = LightPipeline(pipeline_model)

bert_sequence_classifier_patient_complaint download started this may take some time.
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'prediction')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+--------------+
|                                                                                                text|    prediction|
+----------------------------------------------------------------------------------------------------+--------------+
|[My experience in the emergency room was one of the worst of my life. After arriving with severe ...|   [Complaint]|
|[I cannot properly express how much our family appreciates everything the hospital ICU staff did ...|[No_Complaint]|
|[My appointment was not only a complete waste of my time, but the dermatologist's dismissive atti...|   [Complaint]|
|[The nursing staff on the orthopedic ward were exceptional. They explained all my medications and...|[No_Complaint]|
|[I was shocked to receive a bill for over $5,000 after a routine doctor’s appointment because the...|   [Complaint]|
+-------------------------------------------------------

# genericclassifier_patient_complaint_sbiobert_cased_mli

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_patient_complaint_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
genericclassifier_patient_complaint_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'prediction')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+--------------+
|                                                                                                text|    prediction|
+----------------------------------------------------------------------------------------------------+--------------+
|[My experience in the emergency room was one of the worst of my life. After arriving with severe ...|   [Complaint]|
|[I cannot properly express how much our family appreciates everything the hospital ICU staff did ...|[No_Complaint]|
|[My appointment was not only a complete waste of my time, but the dermatologist's dismissive atti...|   [Complaint]|
|[The nursing staff on the orthopedic ward were exceptional. They explained all my medications and...|[No_Complaint]|
|[I was shocked to receive a bill for over $5,000 after a routine doctor’s appointment because the...|   [Complaint]|
+-------------------------------------------------------

# CLASSIFICATION PATIENT URGENCY

## bert_sequence_classifier_patient_urgency

In [4]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

sequenceClassifier = MedicalBertForSequenceClassification\
    .pretrained("bert_sequence_classifier_patient_urgency", "en", "clinical/models")\
    .setInputCols(["document",'token'])\
    .setOutputCol("pred_class")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

bert_sequence_classifier_patient_urgency download started this may take some time.
[OK!]


In [5]:
text_list = [
"""I think my father is having a stroke. His face is drooping, he can’t move his right side and he’s slurring his speech. He is breathing, but it’s really ragged. And, he is not responding when I talk to him…he seems out of it.""",

"""My old neighbor has fallen and cannot get up. She is conscious, but she is in a lot of pain and cannot move.""",

"""My wife has been in pain all morning. She had an operation a few days ago. This morning, she woke up in pain and is having a hard time moving around. The pain is around the surgery area. It is not severe, but it’s making her uncomfortable. She does not have fever, nausea or vomiting. There’s some slight feeling of being bloated.""",

]

In [6]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = model.transform(df)

In [8]:
x = result.select("document.result", "pred_class.result")
df = x.toDF('text', 'prediction')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------+
|                                                                                                text|prediction|
+----------------------------------------------------------------------------------------------------+----------+
|[I think my father is having a stroke. His face is drooping, he can’t move his right side and he’...|    [High]|
|[My old neighbor has fallen and cannot get up. She is conscious, but she is in a lot of pain and ...|  [Medium]|
|[My wife has been in pain all morning. She had an operation a few days ago. This morning, she wok...|     [Low]|
+----------------------------------------------------------------------------------------------------+----------+

